In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/Students/Negin/

/content/drive/My Drive/Colab Notebooks/Students/Negin


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.cluster import adjusted_rand_score as ARI
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
import warnings
warnings.filterwarnings('ignore')
from util import eval, loadDataset, normalization, kmeansInitialization


eps = torch.tensor(10 ** -10)

if torch.cuda.is_available():
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    print("num GPUs", torch.cuda.device_count())
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor
else:
    dtype = torch.FloatTensor
    ltype = torch.LongTensor
    print("CPU")


In [ ]:







datasets = ['3Sources.npy',
'BBCSport.npy',
'Caltech101.npy',
'Caltech_2.npy',
'Citeseer.npy',
'Coil100.npy',
'Cora.npy',
'EYaleB10.npy',
'Handwritten.npy',
'MNIST10.npy',
'UCIdigit.npy',
'Umist.npy',
'Yale32.npy',
'Yeast.npy',
'Cora2.npy',
'texas.npy',
'wisconsin.npy',
'washington.npy',
'cornell.npy']




X0, Y, V, c = loadDataset(10)


r = 40
k = 5
alpha = 1
beta = 10

iter = 300

W = []
H = []
X = []

for v in range(V):

    X.append(torch.tensor(X0[v]).type(dtype))

    d, n = X[v].shape

    W.append(torch.rand(d, r).type(dtype))
    H.append(torch.rand(r, n).type(dtype))

# centering matrix
Q = torch.eye(n).type(dtype) - torch.ones(n,n).type(dtype)/n
Qn = (torch.abs(Q) - Q)/2
Qp = (torch.abs(Q) + Q)/2

A = []
D = []
for v in range(V):

    A0 = kneighbors_graph(X[v].T, k, mode='connectivity', include_self=False).toarray()
    A0 = torch.tensor(A0).type(dtype)
    A0 = torch.maximum(A0, A0.T)

    A.append(A0)
    D.append(torch.diag(torch.sum(A0, dim = 1)))


# Optimization
err = torch.zeros(iter)

for t in range(iter):

    for v in range(V):

        # Updating Ws
        Wn = X[v]        @ H[v].T
        Wd = W[v] @ H[v] @ H[v].T
        W[v] = W[v] * (Wn / torch.maximum(Wd, eps))

        # Updating Hs
        K = torch.zeros(n, n).type(dtype)
        for vv in range(V):
            if vv!=v:
                K += H[v].T @ H[v]

        Hn = W[v].T @ X[v]        + alpha * H[v] @ (Qn @ K @ Qp + Qp @ K @ Qn) + beta * (H[v] @ A[v])
        Hd = W[v].T @ W[v] @ H[v] + alpha * H[v] @ (Qp @ K @ Qp + Qn @ K @ Qn) + beta * (H[v] @ D[v])
        H[v] = H[v] * (Hn / torch.maximum(Hd, eps))

        # Calculating cost function
        # err[t] += torch.norm(X[v] - W[v] @ H[v]) ** 2 + beta * torch.trace(H[v] @ (D[v] - A[v]) @ H[v].T)


# plt.plot(err)


Hfinal = []
for v in range(V):
    Hfinal.append(H[v])


Hfinal = torch.concat(Hfinal, 0)

pred = KMeans(n_clusters=c).fit(Hfinal.T).labels_

nmi, acc, ari, f1mi, f1ma, f1we, pur = eval(Y, pred)
nmi, acc, ari, f1mi, f1ma, f1we, pur

torch.Size([216, 2000]) 3
